In [1]:
"""
GENERAL FUNCTIONS
"""

import csv
import random as rnd

#Function to explore the data in a table

def explore_data(dataset, start = 1, end = 11):
    i = 0
    for s in dataset[0]:
        print('Header', i, '-', s)
        i += 1
    print('\nDataset contains ', len(dataset)-1, ' rows and ',
        len(dataset[0]), ' columns.\n')
    for row in dataset[start:end]:
        print(row)
        print('\n') # adds a new (empty) line after each row
        
#Function to remove rows from a dataset
#dataset = list of rows, rows = list of values to check,
#    column = index of each row to check against,
#    rows_to_keep = whether to keep or remove rows with given values
        
def remove_rows(dataset, rows, column, rows_to_keep = True):
    ret_dataset = [dataset[0]]
    for row in dataset[1:]:
        for s in rows:
            if rows_to_keep:
                if s == row[column]:
                    ret_dataset.append(row)
            else:
                if s != row[column]:
                    ret_dataset.append(row)
            
    return ret_dataset
        
#Function to create a dictionary of table headers

def headers_dictionary(list_headers):
    dict_headers = {}
    i = 0
    for s in list_headers:
        dict_headers[s] = i
        i += 1
    return dict_headers

#Function to wait for the user

def wait():
    npt = input("Press Enter to continue...")
    print('\n')
    
#Function to format the text

def tf(keyword):
    if keyword.lower() == 'b':
        return '\033[1m'
    elif keyword.lower() == 'u':
        return '\033[4m'
    elif keyword.lower() == 'gr':
        return '\033[32m'
    elif keyword.lower() == 'bl':
        return '\033[34m'
    elif keyword.lower() == 'rd':
        return '\033[31m'
    else:
        return '\033[0m'

In [2]:
"""
OTHER GENERAL CODE
"""

#Import data from files

opened_file = open("Game Data - Characters.csv", encoding="utf8")
tbl_characters = list(csv.reader(opened_file))
opened_file = open("Game Data - Cards.csv", encoding="utf8")
tbl_cards = list(csv.reader(opened_file))
opened_file = open("Game Data - Skills.csv", encoding="utf8")
tbl_skills = list(csv.reader(opened_file))

#Establish index dictionaries

cd = headers_dictionary(tbl_cards[0])
sd = headers_dictionary(tbl_skills[0])
sd['max_uses'] = 6
sd['turn_uses'] = 7

#String to print if incorrect input is entered

wrong_input = "Unaccepted value.\n"

#Variables to designate whether to end the game in victory or defeat

#victory = False
#defeat = False

In [3]:
"""
CHARACTER CLASS
"""
class Character():
    def __init__(self, chara_id):
        self.id = chara_id
        chara_dataset = tbl_characters[1:]
        hd = headers_dictionary(tbl_characters[0])
        
        #Find data for this character from the dataset
        chara_data = []
        for row in chara_dataset:
            if self.id == 'nothing':
                for x in row:
                    chara_data.append('0')
            if self.id == row[hd['id']]:
                for x in row:
                    chara_data.append(x)
        if chara_data == []:
            print('ERROR: Character does not exist!')
                
        #Initialize stats
        self.name = tf('b') + chara_data[hd['name']] + tf('')
        self.type = chara_data[hd['type']]
        self.max_hp = int(chara_data[hd['health']])
        self.hp = int(chara_data[hd['health']])
        self.at = int(chara_data[hd['attack']])
        self.df = int(chara_data[hd['defense']])
        self.ag = int(chara_data[hd['agility']])
        self.defeated = False
        
        self.deck = []
        self.hand = []
        self.discard = []
        self.skills = []
        
        #Heroes
        if self.type == 'Hero':
            self.setup_hero()
            
        #Non-heroes
        else:
            self.ap = 0
            
    def setup_hero(self):
        self.ap = 100
        
        #Setup deck
        for card in tbl_cards[1:]:
            if card[cd['hero_id']] == self.id:
                self.deck.append(card)
        rnd.shuffle(self.deck)
        
        #Draw starting cards
        for card in tbl_cards[1:]:
            if 'generic' in card[cd['card_id']]:
                self.hand.append(card)
                
        #Setup skills
        for skill in tbl_skills[1:]:
            my_skill = []
            for f in skill:
                my_skill.append(f)
            if my_skill[sd['hero_id']] == self.id or 'generic' in my_skill[sd['skill_id']]:
                self.skills.append(my_skill)
        
        #Learn starting skills
        i = 0
        for skill in self.skills:
            if 'generic' in skill[sd['skill_id']]:
                self.skills[i].append(1)
                self.skills[i].append(1)
            else:
                self.skills[i].append(0)
                self.skills[i].append(0)
            i += 1
            
    def attack(self, target):
        dmg = self.at
        print(self.name, 'attacks', target.name)
        damage({'attacker': self, 'target': target, 'damage_dealt': dmg})
        
    def draw_card(self):
        
        if self.deck == []:
            print('There are no cards in your deck. Shuffling discard into deck...')
            if self.discard == []:
                print('There are no cards in your discard. Unable to draw a card.')
                return
            else:
                self.deck = self.discard
                rnd.shuffle(self.deck)
                self.discard = []
        
        new_card = self.deck[0]
        self.hand.append(new_card)
        self.deck.remove(new_card)
        print(self.name, 'drew a card:', tf('b'), new_card[cd['card_name']], tf(''))
        print('\n')
        
    def learn_skill(self, new_skill):
        if new_skill[sd['max_uses']] == 0:
            new_skill[sd['max_uses']] = 1
            new_skill[sd['turn_uses']] += 1
            print(self.name, 'learned a skill:', tf('b'), new_skill[sd['skill_name']], tf(''))
        else:
            new_skill[sd['max_uses']] += 1
            new_skill[sd['turn_uses']] += 1
            print(self.name, 'leveled up a skill:', tf('b'), new_skill[sd['skill_name']], tf(''))
        print('\n')
        
    def heal(self, amount):
        original_hp = self.hp
        self.hp += amount
        if self.hp > self.max_hp:
            self.hp = self.max_hp
        if self.hp > original_hp:
            print(self.hp - original_hp, 'HP was restored to', self.name + '.')
            #wait()
        else:
            print(self.name, 'is already at max HP.')
            #wait()
            
    def list_stats(self):
        print(self.name, '--', self.type)
        print('HP:', self.hp, '-- Atk:', self.at, '-- Def:', self.df, '-- Agility:', self.ag)
        print('\n')
        
    def play(self):
        if self.hp <= 0:
            print(self.name, 'is defeated.')
            wait()
            return False
        play_again = True
        usable_skills = []
        for skill in self.skills:
            if skill[sd['turn_uses']] > 0:
                usable_skills.append(skill)
                
        #List stats
        self.list_stats()
                
        #List skills
        print(tf('b') + tf('u') + tf('gr') + '***   SKILLS   ***\n' + tf(''))
        i = 0
        for skill in usable_skills:
            """
            print(tf('b') + tf('gr') + 's' + str(i) + ':' + tf(''),
                tf('b') + skill[sd['skill_name']] + tf(''), '-', skill[sd['gameplay']])
            print('Action Points required:', tf('rd'), skill[sd['action_points']], tf(''), '/', self.ap)
            print('Uses left this turn:', tf('rd'), skill[sd['turn_uses']], tf(''))
            print('\n')
            """
            if int(skill[sd['action_points']]) > self.ap:
                ap_color = 'rd'
            else:
                ap_color = 'gr'
                
            print(tf('b') + tf('gr') + 's' + str(i) + ':' + tf(''),
                  tf('b') + skill[sd['skill_name']] + tf(''),
                  '-- AP required:', tf('b'), tf(ap_color), skill[sd['action_points']], tf(''),
                  '/', tf('b'), self.ap, tf(''),
                  '-- Uses left this turn:', tf('b'), skill[sd['turn_uses']], tf(''))
            i += 1
        print('\n')
            
        #List cards
        print(tf('b') + tf('u') + tf('bl') + '***   CARDS   ***\n' + tf(''))
        i = 0
        for card in self.hand:
            """
            print(tf('b') + tf('bl') + 'c' + str(i) + ':' + tf(''),
                  tf('b') + card[cd['card_name']], tf(''), '-', card[cd['gameplay']])
            print('Action Points required:', tf('rd'), card[cd['action_points']], tf(''), '/', self.ap)
            print('\n')
            """
            if int(card[cd['action_points']]) > self.ap:
                ap_color = 'rd'
            else:
                ap_color = 'gr'
                
            print(tf('b') + tf('bl') + 'c' + str(i) + ':' + tf(''),
                  tf('b') + card[cd['card_name']], tf(''),
                  '-- AP required:', tf('b'), tf(ap_color), card[cd['action_points']], tf(''),
                  '/', tf('b'), self.ap, tf(''))
            i += 1
        print('\n')
        
        
        while True:
            #Allow player to choose a card or skill
            npt = input('Type ' + tf('gr') + 's ' + tf('') + 'for ' + tf('gr') + 'skill ' + tf('')
                        + 'or ' + tf('bl') + 'c ' + tf('') + 'for ' + tf('bl') + 'card' + tf('')
                        + ', and then the number of the one you want to use.'
                        + '\nType e to end ' + self.name + "'s turn.")
            print('\n')
        
        
            #If the player chooses a skill
            if npt.lower().startswith('s'):
                try:
                    npt = int(npt.lstrip('s'))
                except ValueError:
                    print(wrong_input)
                    continue
                try:
                    skill_used = usable_skills[npt]
                except IndexError:
                    print(wrong_input)
                    continue
                    
                for skill in self.skills:
                    if skill[sd['skill_id']] == skill_used[sd['skill_id']]:
                        if use_skill(self, skill):
                            skill[sd['turn_uses']] -= 1
                wait()
                break
            
            
            #If the player chooses a card
            elif npt.lower().startswith('c'):
                try:
                    npt = int(npt.lstrip('c'))
                except ValueError:
                    print(wrong_input)
                    continue
                try:
                    card_played = self.hand[npt]
                except IndexError:
                    print(wrong_input)
                    continue
                """
                req_ap = int(card_played[cd['action_points']])
                if self.ap < req_ap:
                    print("You don't have enough Action Points.")
                    continue
                """
                #print('You played a ' + tf('bl') + 'card:' + tf(''), card_played[cd['card_name']])
                
                if play_card(self, card_played):
                    self.discard.append(card_played)
                    del self.hand[npt]
                wait()
                break
            
            
            #If the player ends the turn
            elif npt.lower().startswith('e'):
                play_again = False
                break
                
                
            #Other input for testing
            elif npt.lower().startswith('t'):
                if npt.lower() == 'tsd':
                    print('You dead.')
                    damage({'target': self, 'damage_dealt': 9001})
                    break
                if npt.lower() == 'tdb':
                    print('Boss dead.')
                    damage({'target': boss.enemy, 'damage_dealt': 999999999999})
                    break
                if npt.lower() == 'tdc':
                    print('You drew like a bazillion cards. Or maybe just 5.')
                    self.draw_card()
                    self.draw_card()
                    self.draw_card()
                    self.draw_card()
                    self.draw_card()
                    break
                if npt.lower() == 'tls':
                    print('Now you got skillz.')
                    skillz = choose(get_skill_list(self, False))
                    if skillz != False:
                        self.learn_skill(skillz)
                    break
                    
                
            else:
                print(wrong_input)
                
        if self.defeated or boss.enemy.defeated:
            play_again = False
            
        return play_again

In [4]:
"""
TESTBOSS CLASS
"""
class TestBoss():
    def __init__(self):
        self.defeated = False
        self.enemy = Character('general_zook')
        
    def start_game(self):
        characters.append(self.enemy)
        print("Test Boss emerges!")
        print("Will the heroes pass the test?")
        wait()
        
    def boss_turn(self):
        self.enemy.list_stats()
        print("Test Boss attacks again!")
        damage({'target': self.pick_rand_hero(), 'attacker': self.enemy})
        #attack(self.enemy, self.pick_rand_hero())
        wait()
        
    def after_turn(self):
        print("Test Boss attacks!")
        damage({'target': self.pick_rand_hero(), 'attacker': self.enemy})
        #attack(self.enemy, self.pick_rand_hero())
        wait()
        
    def pick_rand_hero(self):
        hero_list = get_hero_list()
        ret_hero = rnd.choice(hero_list)
        return ret_hero

In [5]:
"""
MISC GAME FUNCTIONS
"""

def turn(chara):
    print('\n')
    print('It is now', chara.name + "'s turn!")
    print('\n')
    wait()
    
    if chara.type == 'Hero':
        chara.ap = 100
        for skill in chara.skills:
            skill[sd['turn_uses']] = skill[sd['max_uses']]
    
        while chara.play():
            print('\n')
            continue
    
    if chara.type == 'Boss':
        boss.boss_turn()
        
    
    #return chara, quit


def get_chara_list(chara_type):
    ret_list = []
    for chara in characters:
        if chara.type == chara_type:
            ret_list.append(chara)
            
    return ret_list


def get_hero_list():
    ret_list = get_chara_list('Hero') + get_chara_list('Summon')
    return ret_list


def get_enemy_list():
    ret_list = get_chara_list('Boss') + get_chara_list('Minion')
    return ret_list


def get_skill_list(hero, learned):
    ret_list = []
    for skill in hero.skills:
        if learned:
            if skill[sd['max_uses']] > 0:
                ret_list.append(skill)
        else:
            if skill[sd['max_uses']] == 0:
                ret_list.append(skill)
                
    return ret_list


def choose(obj_list):
    if obj_list == []:
        return False
    print('\n')
    i = 1
    for obj in obj_list:
        if type(obj) == Character:
            chara = obj
            print(str(i) + ':')
            chara.list_stats()
            #print(str(i) + ':', chara.name, '--', chara.type)
            #print('HP:', chara.hp, '-- Atk:', chara.at, '-- Def:', chara.df, '-- Agility:', chara.ag)
            #print('\n')
        if type(obj) == list:
            if len(obj) == 8:
                skill = obj
                print(str(i) + ':', tf('b'), skill[sd['skill_name']], tf(''), '-', skill[sd['gameplay']])
            if len(obj) == 5:
                card = obj
                print(str(i) + ':', tf('b'), card[cd['card_name']], tf(''), '-', card[cd['gameplay']])
        
        i += 1
            
    print('\n')
    
    while True:
        npt = input('Type a number to make a choice:')
        print('\n')
        try:
            npt = int(npt)
        except ValueError:
            print(wrong_input)
            continue
        if npt < 1:
            print(wrong_input)
            continue
        try:
            selection = obj_list[npt - 1]
        except IndexError:
            print(wrong_input)
            continue
            
        return selection
        
    return None


def damage(damage_mods):
    dm = damage_mods
    if 'target' in dm:
        target = dm['target']
    else:
        print('ERROR: No target selected!')
        wait()
        return
    if 'attacker' in dm:
        attacker = dm['attacker']
    else:
        attacker = Character('nothing')
    if 'damage_dealt' in dm:
        damage_dealt = dm['damage_dealt']
    else:
        damage_dealt = attacker.at
    
    damage_taken = damage_dealt - target.df
    if damage_taken > 0:
        target.hp -= damage_taken
        if 'attacker' in dm:
            print(attacker.name, 'dealt', damage_taken, 'damage to', target.name + '!')
        else:
            print(damage_taken, 'damage was dealt to', target.name + '!')
        #wait()
    else:
        print(target.name, 'prevented all damage.')
        #wait()
        
    if target.hp <= 0:
        target.hp = 0
        target.defeated = True

In [6]:
"""
GENERIC SKILLS
"""

def use_skill(user, skill):
    skill_id = skill[sd['skill_id']]
    skill_name = skill[sd['skill_name']]
    req_ap = int(skill[sd['action_points']])
    
    if user.ap < req_ap:
        print(user.name, "doesn't have enough Action Points to use that skill.")
        return False
    else:
        print('You selected a ' + tf('gr') + 'skill:' + tf(''), tf('b') + skill_name + tf(''))
        print(skill[sd['gameplay']])
        print('Action Points required:', tf('gr'), req_ap, tf(''), '/', tf('b'), user.ap, tf(''))
        print('Uses left this turn:', tf('b'), skill[sd['turn_uses']], tf(''))
        print('\n')
        
        npt = input('Press Enter to use this skill, or type something else to cancel:')
        print('\n')
        if npt == '':
            print(user.name, 'used a ' + tf('gr') + 'skill:' + tf(''), tf('b') + skill_name + tf(''))
            user.ap -= req_ap
        else:
            return False
    
    
    # Attack
    if skill_id == 'generic_s1':
        enemy = choose(get_enemy_list())
        user.attack(enemy)
        return True
    
    # Prepare
    if skill_id == 'generic_s2':
        user.draw_card()
        return True
    
    # Rest
    if skill_id == 'generic_s3':
        user.heal(2)
        return True
    
    
    print('This skill has not been created yet, so nothing happens.')
    return True


"""
GENERIC CARDS
"""
    
def play_card(user, card):
    card_id = card[cd['card_id']]
    card_name = card[cd['card_name']]
    req_ap = int(card[cd['action_points']])

    if user.ap < req_ap:
        print(user.name, "doesn't have enough Action Points to play that card.")
        return False
    else:
        print('You selected a ' + tf('bl') + 'card:' + tf(''), tf('b') + card_name + tf(''))
        print(card[cd['gameplay']])
        print('Action Points required:', tf('gr'), req_ap, tf(''), '/', tf('b'), user.ap, tf(''))
        print('\n')
        
        npt = input('Press Enter to play this card, or type something else to cancel:')
        print('\n')
        if npt == '':
            print(user.name, 'played a ' + tf('bl') + 'card:' + tf(''), tf('b') + card_name + tf(''))
            user.ap -= req_ap
        else:
            return False
        
    
    # Double Attack
    if card_id == 'generic_c1':
        enemy = choose(get_enemy_list())
        user.attack(enemy)
        user.attack(enemy)
        return True
    
    # Study
    if card_id == 'generic_c2':
        skill = choose(get_skill_list(user, False))
        if skill != False:
            user.learn_skill(skill)
        else:
            print('There are no new skills to learn.')
        return True
    
    # Strategize
    if card_id == 'generic_c3':
        user.draw_card()
        user.draw_card()
        return True
    
        
    print('This card has not been created yet, so nothing happens.')
    return True

In [7]:
"""
INITIALIZE NEW GAME
"""

"""
HEROES:
    
lady_lynne = Lady Lynne
mecha_man = Mecha-Man
boss_kuza = Boss Kuza
mrs_hagglin = Mrs. Hagglin
"""

#Heroes in new game
str_heroes = ['lady_lynne', 'mecha_man', 'boss_kuza', 'mrs_hagglin']

"""
BOSSES:

TestBoss() = Test Boss
"""

#Boss in new game
boss = TestBoss()


#Create the Hero objects
characters = []

for chara_id in str_heroes:
    characters.append(Character(chara_id))



"""
RUN GAME
"""
print(input("Press Enter to start the game!"))
print('\n')

quit = False

boss.start_game()

round_num = 1
while not quit:
    print('\n')
    print(tf('b') + "Round", round_num, tf(''))
    wait()
    print('\n')
    total_characters = len(characters)
    current_character = 0
    while current_character < total_characters:
        #Do the character's turn
        turn(characters[current_character])
            
        #Delete defeated characters
        i = 0
        chara_list = characters
        for chara in chara_list:
            if chara.defeated:
                print(tf('rd') + chara.name, tf('rd') + 'has been defeated!' + tf(''))
                del characters[i]
                current_character -= 1
            i += 1
            
        #End the game if heroes win or lose
        if get_hero_list() == []:
            print('The heroes have all been defeated.')
            wait()
            quit = True
            break
        if boss.enemy.defeated:
            print('The heroes are victorious!')
            wait()
            quit = True
            break
                
        #Check if the player wants to quit
        npt = input('Press q to quit. Press Enter to continue.')
        print('\n')
        if npt == 'q' or npt == 'Q':
            quit = True
            break
        else:
            boss.after_turn()
                
        #Go to the next character
        total_characters = len(characters)
        current_character += 1
    
    round_num += 1
        
    """
    for chara in characters:
        if not quit:
            chara, quit = turn(chara)
            
            #End of the turn
            i = 0
            for chara in characters:
                if chara.defeated:
                    print(tf('rd') + chara.name, tf('rd') + 'has been defeated!' + tf(''))
                    characters.insert(i, Character('boss_kuza'))
                    #del characters[i]
                i += 1
        
            if get_hero_list() == []:
                print('The heroes have all been defeated.')
                wait()
                quit = True
                break
            if boss.defeated:
                print('The heroes are victorious!')
                wait()
                quit = True
                break
    
            npt = input('Press q to quit. Press Enter to continue.')
            print('\n')
            if npt == 'q' or npt == 'Q':
                quit = True
            else:
                quit = False
                
            print('\n')
            
            if not quit:
                boss.after_turn()
    round_num += 1
    """

print('You have quit the game.')

Press Enter to start the game!



Test Boss emerges!
Will the heroes pass the test?
Press Enter to continue...




Round 1 
Press Enter to continue...






It is now Lady Lynne's turn!


Press Enter to continue...


Lady Lynne -- Hero
HP: 100 -- Atk: 2 -- Def: 0 -- Agility: 5


***   SKILLS   ***

s0: Attack -- AP required:   50  /  100  -- Uses left this turn:  1 
s1: Prepare -- AP required:   20  /  100  -- Uses left this turn:  1 
s2: Rest -- AP required:   30  /  100  -- Uses left this turn:  1 


***   CARDS   ***

c0: Double Attack  -- AP required:   80  /  100 
c1: Study  -- AP required:   50  /  100 
c2: Strategize  -- AP required:   50  /  100 


Type s for skill or c for card, and then the number of the one you want to use.
Type e to end Lady Lynne's turn.tdc


You drew like a bazillion cards. Or maybe just 5.
Lady Lynne drew a card:  Shield Training 


Lady Lynne drew a card:  Weapon Training 


Lady Lynne drew a card:  Light Shield 


Lady Lynne drew a card:  Careful Explo

Type s for skill or c for card, and then the number of the one you want to use.
Type e to end Lady Lynne's turn.tls


Now you got skillz.


1:  Discover Weapon  - Add a random Weapon card from your deck to your hand.
2:  Discover Shield  - Add a random Shield card from your deck to your hand.


Type a number to make a choice:2


Lady Lynne learned a skill:  Discover Shield 




Lady Lynne -- Hero
HP: 100 -- Atk: 2 -- Def: 0 -- Agility: 5


***   SKILLS   ***

s0: Discover Shield -- AP required:   20  /  100  -- Uses left this turn:  1 
s1: Special Attack -- AP required:   20  /  100  -- Uses left this turn:  1 
s2: Defend -- AP required:   20  /  100  -- Uses left this turn:  1 
s3: Attack -- AP required:   50  /  100  -- Uses left this turn:  1 
s4: Prepare -- AP required:   20  /  100  -- Uses left this turn:  1 
s5: Rest -- AP required:   30  /  100  -- Uses left this turn:  1 


***   CARDS   ***

c0: Double Attack  -- AP required:   80  /  100 
c1: Study  -- AP required:   50  /  



Press q to quit. Press Enter to continue.


Test Boss attacks!
General Zook dealt 10 damage to Boss Kuza!
Press Enter to continue...




It is now Mecha-Man's turn!


Press Enter to continue...


Mecha-Man -- Hero
HP: 105 -- Atk: 4 -- Def: 2 -- Agility: 6


***   SKILLS   ***

s0: Attack -- AP required:   50  /  100  -- Uses left this turn:  1 
s1: Prepare -- AP required:   20  /  100  -- Uses left this turn:  1 
s2: Rest -- AP required:   30  /  100  -- Uses left this turn:  1 


***   CARDS   ***

c0: Double Attack  -- AP required:   80  /  100 
c1: Study  -- AP required:   50  /  100 
c2: Strategize  -- AP required:   50  /  100 


Type s for skill or c for card, and then the number of the one you want to use.
Type e to end Mecha-Man's turn.tdc


You drew like a bazillion cards. Or maybe just 5.
Mecha-Man drew a card:  "Bunker Mode, Activate!" 


Mecha-Man drew a card:  "Cycle Mode, Activate!" 


Mecha-Man drew a card:  "Tank Mode, Activate!" 


There are no cards in your deck. Sh



Mrs. Hagglin -- Hero
HP: 92 -- Atk: 1 -- Def: 0 -- Agility: 3


***   SKILLS   ***

s0: Attack -- AP required:   50  /  100  -- Uses left this turn:  1 
s1: Prepare -- AP required:   20  /  100  -- Uses left this turn:  1 
s2: Rest -- AP required:   30  /  100  -- Uses left this turn:  1 


***   CARDS   ***

c0: Double Attack  -- AP required:   80  /  100 
c1: Study  -- AP required:   50  /  100 
c2: Strategize  -- AP required:   50  /  100 


Type s for skill or c for card, and then the number of the one you want to use.
Type e to end Mrs. Hagglin's turn.


Unaccepted value.

Type s for skill or c for card, and then the number of the one you want to use.
Type e to end Mrs. Hagglin's turn.tdb


Boss dead.
999999999995 damage was dealt to General Zook!
General Zook has been defeated!
The heroes are victorious!
Press Enter to continue...


You have quit the game.
